<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Загрузка-предобученной-модели-BERT" data-toc-modified-id="Загрузка-предобученной-модели-BERT-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка предобученной модели BERT</a></span></li><li><span><a href="#Создание-признаков-для-модели" data-toc-modified-id="Создание-признаков-для-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Создание признаков для модели</a></span></li><li><span><a href="#Обучение-и-тестирование-моделей" data-toc-modified-id="Обучение-и-тестирование-моделей-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обучение и тестирование моделей</a></span></li></ul></div>

# Определение токсичности комментариев для "Викишоп"





Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Требуется значение метрики качества *F1* не меньше 0.75. 


**Описание данных**

_text_ - текст комментария;  
_toxic_ - целевой признак (1-комментарий токсичен, 0-нетоксичен)

## Подготовка данных

In [1]:
!pip install transformers

In [2]:
import numpy as np
import pandas as pd
import torch
import transformers

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score

from tqdm import notebook
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

data.sample(5)

,text,toxic
141421,Yeah mate I can see where you're coming from b...,0
108700,GV descendants AFD \n\nWikipedia:Articles_for_...,0
74877,"List of works by William Monahan\nMBK, do you ...",0
94479,Dude this is crazy!!!!,0
85510,"No worries, it's understandable considering th...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Датафрейм содержит 159571 строку. Столбец "text" типа object содержит текст комментария, а столбец "toxic" типа int64 содержит метку токсичности комментария (1-комментарий токсичен, 0-комментарий нетоксичен). Перед нами задача бинарной классификации. Проверим, как соотносятся классы между собой:

In [5]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Видим, что количество токсичных комментариев составляет всего 10% от общего числа комментариев. С одной стороны, радует, что и в интернете люди стараются общаться культурно. Но с другой стороны, мы имеем дело с сильным дисбалансом классов, что может сказаться на качестве обучения наших моделей. 

В целях экономии оперативной памяти и ускорения работы возьмем только часть датасета - 10000 строк. Если получится приемлемая метрика качества, значит, мы справились c заданием.

In [6]:
data_sample = data.sample((10000), random_state=21).reset_index(drop=True) 
data_sample['toxic'].value_counts()

0    9007
1     993
Name: toxic, dtype: int64

Видим, что в новом урезанном датасете доля токсичных комментариев около 10%, как и в полном датасете.

## Загрузка предобученной модели BERT

Воспользуемся уменьшенной версией BERT - DistilBert. Она немного уступает в точности, но работает быстрее. Алгоритм работы будет такой:

- на предобученной модели DistilBert выполним токенизацию текстов и создадим эмбеддинги;
- полученные эмбеддинги отправим в другую модель (например, логистическую регрессию) для непосредственно классификации. 


In [7]:
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Выполним токенизацию датасета:

In [8]:
tokenized = data_sample['text'].apply((lambda x: tokenizer.encode(x, max_length=512, truncation=True, add_special_tokens=True)))


Для работы модели DistilBERT необходимо, чтобы длина текстов в корпусе была одинаковой. Найдем длину наибольшего во всем датасете вектора; более короткие векторы дополним нулями.

In [9]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape

(10000, 512)

Теперь поясним модели, что нули не несут значимой информации. Создадим маску для важных токенов:

In [10]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(10000, 512)

## Создание признаков для модели

Создадим эмбеддинги батчами небольшого размера, чтобы хватило оперативной памяти, и пройдемся циклом по батчам.

In [11]:
batch_size = 200
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

In [12]:
features = np.concatenate(embeddings)
target = data_sample['toxic']

In [13]:
features.shape

(10000, 768)

Разделим данные на обучающую и тестовую выборку. Размер тестовой выборки - 15% данных.

In [14]:
train_features, test_features, train_target, test_target = train_test_split(features, target, stratify=target,  test_size=0.15, random_state=25)

In [15]:
train_features.shape

(8500, 768)

In [16]:
test_features.shape

(1500, 768)

Признаки готовы, можно приступать к обучению моделей.

## Обучение и тестирование моделей

Рассмотрим несколько моделей: LogisticRegression, KNeighborsClassifier, SVC. Гиперпараметры подбирать не будем, в данном проекте это занимает много времени и не дает увеличения метрики.

In [17]:
model = LogisticRegression(random_state=21)
model.fit(train_features, train_target)
pred = model.predict(test_features)
f1_log = f1_score(test_target, pred)
print('F1 LogisticRegression: ', round(f1_log, 2))

F1 LogisticRegression:  0.79


In [18]:
model = KNeighborsClassifier()
model.fit(train_features, train_target)
pred = model.predict(test_features)
f1_knn = f1_score(test_target, pred)
print('F1 KNeighborsClassifier: ', round(f1_knn, 2))

F1 KNeighborsClassifier:  0.66


In [19]:
model = SVC()
model.fit(train_features, train_target)
pred = model.predict(test_features)
f1_svc = f1_score(test_target, pred)
print('F1 SVC: ', round(f1_svc, 2))

F1 SVC:  0.69


Сравним наши метрики с простой моделью, заполненной случайно:

In [21]:
model = DummyClassifier(strategy = 'stratified')

model.fit(train_features, train_target)
pred = model.predict(test_features)
f1_dummy = f1_score(test_target, pred)
print('F1 Dummy: ', round(f1_dummy, 2))

F1 Dummy:  0.08


Взглянем на итоговую таблицу метрик:

In [22]:
data = {'F1-score':[f1_log, f1_knn, f1_svc, f1_dummy]}

table = pd.DataFrame(data)
table.index = ['LogisticRegression', 'KNeighborsClassifier','SVC', 'Dummy']

pd.set_option('display.float_format', '{:.2f}'.format)
display(table)

,F1-score
LogisticRegression,0.79
KNeighborsClassifier,0.66
SVC,0.69
Dummy,0.08


**Общий вывод:** несмотря на то, что мы взяли уменьшенный объем выборки (10000 строк), а также несмотря на несбалансированность классов, нам удалось достичь требуемой метрики при помощи логистической регрессии. KNeighborsClassifier и SVC показали результат хуже. Но любая наша модель пресказывает лучше дамми-модели, и это успех.